In [ ]:
import requests
import bs4
import pandas as pd

In [27]:
# URL della pagina
url_province = 'https://italia.indettaglio.it/ita/province/province.html'
url_regioni = 'https://italia.indettaglio.it/ita/regioni/regioni.html'

# Eseguire la richiesta HTTP
requisicao = requests.get(url_province, verify=False)
pagina_province = bs4.BeautifulSoup(requisicao.text, "html.parser")

# Eseguire la richiesta HTTP
requisicao = requests.get(url_regioni, verify=False)
pagina_regioni = bs4.BeautifulSoup(requisicao.text, "html.parser")

c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [28]:
informazioni_geografiche = pd.read_csv("informazioni_geografiche.csv")
informazioni_geografiche.head()

,sigla_provincia,Comune,Provincia,Regione,Provincie nella regione (N°),Comuni nella regione (N°),Superficie (kmq) della Regione,Macro Zona della Regione,Zona Precisa della Regione
0,TO,Agliè,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
1,TO,Airasca,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
2,TO,Ala di Stura,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
3,TO,Albiano d'Ivrea,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
4,TO,Almese,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest


## Informazioni Regionali

In [29]:
# Trova tutte le tabelle nella pagina
tabelle = pagina_regioni.find_all("table", class_="table")

# Assicuriamoci di selezionare la tabella giusta (quella con 5 colonne)
tabella_popolazione = None
for tabella in tabelle:
    if len(tabella.find_all("tr")[0].find_all("td")) == 5:
        tabella_popolazione = tabella
        break

# Se la tabella non è stata trovata, interrompiamo
if tabella_popolazione is None:
    print("Tabella della popolazione non trovata.")
else:
    # Liste per salvare i dati
    province = []
    pop_maschile = []
    pop_femminile = []
    pop_totale = []
    ultracentenari = []

    # Itera sulle righe della tabella
    for riga in tabella_popolazione.find_all("tr")[1:]:  # Salta la prima riga (intestazioni)
        colonne = riga.find_all("td")
        if len(colonne) == 5:
            provincia = colonne[0].text.strip()
            maschi = colonne[1].text.strip().replace(".", "")
            femmine = colonne[2].text.strip().replace(".", "")
            totale = colonne[3].text.strip().replace(".", "")
            centenari = colonne[4].text.strip().replace(".", "")

            province.append(provincia)
            pop_maschile.append(int(maschi))
            pop_femminile.append(int(femmine))
            pop_totale.append(int(totale))
            ultracentenari.append(int(centenari))

    # Creare un DataFrame con i dati
    informazioni_regioni = pd.DataFrame({
        "Regione": province,
        "Popolazione Maschile": pop_maschile,
        "Popolazione Femminile": pop_femminile,
        "Popolazione Totale": pop_totale,
        "Ultracentenari": ultracentenari
    })

In [30]:
regioni_italiane = {
    "Piemonte": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Valle d'Aosta": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Lombardia": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Liguria": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Veneto": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Trentino-Alto Adige": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Friuli-Venezia Giulia": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Emilia-Romagna": {"macro_area": "Nord", "dettagliata": "Nord"},
    "Toscana": {"macro_area": "Centro", "dettagliata": "Centro"},
    "Umbria": {"macro_area": "Centro", "dettagliata": "Centro"},
    "Marche": {"macro_area": "Centro", "dettagliata": "Centro-Est"},
    "Lazio": {"macro_area": "Centro", "dettagliata": "Centro-Sud"},  # Lazio ha due posizioni
    "Abruzzo": {"macro_area": "Sud", "dettagliata": "Centro-Sud"},
    "Molise": {"macro_area": "Sud", "dettagliata": "Centro-Sud"},
    "Campania": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Puglia": {"macro_area": "Sud", "dettagliata": "Sud-Est"},
    "Basilicata": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Calabria": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Sicilia": {"macro_area": "Sud", "dettagliata": "Sud-Ovest"},
    "Sardegna": {"macro_area": "Ovest", "dettagliata": "Ovest"}
}

print(regioni_italiane["Basilicata"])

{'macro_area': 'Sud', 'dettagliata': 'Sud'}


In [31]:
def imposta_macro_area(regione):
    return regioni_italiane[regione]["macro_area"]

informazioni_regioni["Macro Zona della Regione"] = informazioni_regioni["Regione"].apply(imposta_macro_area)

def imposta_dettagliata(regione):
    return regioni_italiane[regione]["dettagliata"]

informazioni_regioni["Zona Precisa della Regione"] = informazioni_regioni["Regione"].apply(imposta_dettagliata)

In [32]:
informazioni_regioni

,Regione,Popolazione Maschile,Popolazione Femminile,Popolazione Totale,Ultracentenari,Macro Zona della Regione,Zona Precisa della Regione
0,Abruzzo,647812,683762,1331574,467,Sud,Centro-Sud
1,Basilicata,282525,294094,576619,156,Sud,Sud
2,Calabria,964960,1011671,1976631,629,Sud,Sud
3,Campania,2848043,3013486,5861529,1120,Sud,Sud
4,Emilia-Romagna,2155222,2295286,4450508,1874,Nord,Nord
5,Friuli-Venezia Giulia,594007,633115,1227122,586,Nord,Nord-Est
6,Lazio,2841686,3050739,5892425,1612,Centro,Centro-Sud
7,Liguria,752645,830618,1583263,844,Nord,Nord-Ovest
8,Lombardia,4881615,5121000,10002615,3055,Nord,Nord-Ovest
9,Marche,750875,799921,1550796,580,Centro,Centro-Est


In [33]:
informazioni_regioni.to_csv("informazioni_regioni.csv", index=False)

## Informazioni Provinciali

In [34]:
# Trova tutte le tabelle nella pagina
tabelle = pagina_province.find_all("table", class_="table")

# Assicuriamoci di selezionare la tabella giusta (quella con 5 colonne)
tabella_popolazione = None
for tabella in tabelle:
    if len(tabella.find_all("tr")[0].find_all("td")) == 5:
        tabella_popolazione = tabella
        break

# Se la tabella non è stata trovata, interrompiamo
if tabella_popolazione is None:
    print("Tabella della popolazione non trovata.")
else:
    # Liste per salvare i dati
    province = []
    pop_maschile = []
    pop_femminile = []
    pop_totale = []
    ultracentenari = []

    # Itera sulle righe della tabella
    for riga in tabella_popolazione.find_all("tr")[1:]:  # Salta la prima riga (intestazioni)
        colonne = riga.find_all("td")
        if len(colonne) == 5:
            provincia = colonne[0].text.strip()
            maschi = colonne[1].text.strip().replace(".", "")
            femmine = colonne[2].text.strip().replace(".", "")
            totale = colonne[3].text.strip().replace(".", "")
            centenari = colonne[4].text.strip().replace(".", "")

            province.append(provincia)
            pop_maschile.append(int(maschi))
            pop_femminile.append(int(femmine))
            pop_totale.append(int(totale))
            ultracentenari.append(int(centenari))

    # Creare un DataFrame con i dati
    informazioni_provinciali = pd.DataFrame({
        "Provincia": province,
        "Popolazione Maschile": pop_maschile,
        "Popolazione Femminile": pop_femminile,
        "Popolazione Totale": pop_totale,
        "Ultracentenari": ultracentenari
    })

print(informazioni_provinciali.shape[0])
informazioni_provinciali.head()

107


,Provincia,Popolazione Maschile,Popolazione Femminile,Popolazione Totale,Ultracentenari
0,Agrigento,217412,230326,447738,107
1,Alessandria,208164,223721,431885,228
2,Ancona,230229,247663,477892,169
3,Aosta,62564,65734,128298,38
4,Arezzo,168004,178438,346442,126


In [35]:
informazioni_provinciali.to_csv("informazioni_provinciali.csv", index=False)